<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%ED%86%B5%ED%95%A9/(%ED%86%B5%ED%95%A9)_EDA_%EA%B3%BC%EC%A0%95_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 13.5 MB/s eta 0:00:00


In [2]:
import pickle
import random
from konlpy.tag import Okt

# 동의어 사전 로드
def load_synonym_dict(pickle_file):
    with open(pickle_file, 'rb') as f:
        synonym_dict = pickle.load(f)
    return synonym_dict

# Random Swap (랜덤 단어 교환)
def random_swap(words, n=1):
    words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# Random Insertion (랜덤 단어 삽입)
def random_insertion(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words.insert(idx, random.choice(synonym_dict[word]))
    return words

# Random Delete (랜덤 단어 삭제)
def random_delete(words, p=0.1):
    if len(words) == 1:
        return words
    return [word for word in words if random.random() > p]

# Synonym Replacement (동의어 대체)
def synonym_replacement(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words[idx] = random.choice(synonym_dict[word])
    return words

# 데이터 증강 실행 함수
#def augment_text(text, synonym_dict, swap_n=1, insert_n=1, delete_p=0.1, replace_n=1):
#    okt = Okt()
#    words = okt.morphs(text)  # Okt를 이용한 토큰화
#    words = random_swap(words, swap_n)
#    words = random_insertion(words, synonym_dict, insert_n)
#    words = random_delete(words, delete_p)
#    words = synonym_replacement(words, synonym_dict, replace_n)
#    return ' '.join(words)

In [4]:
# 동의어 사전 로드
synonym_dict = load_synonym_dict('/content/wordnet.pickle')

In [5]:
# 데이터 증강 실행 함수
def augment_text(text, synonym_dict, swap_n=0, insert_n=0, delete_p=0, replace_n=0):
    okt = Okt()
    words = okt.morphs(text)  # Okt를 이용한 토큰화

    if swap_n > 0:
        words = random_swap(words, swap_n)
    if insert_n > 0:
        words = random_insertion(words, synonym_dict, insert_n)
    if delete_p > 0:
        words = random_delete(words, delete_p)
    if replace_n > 0:
        words = synonym_replacement(words, synonym_dict, replace_n)

    return ' '.join(words)

# 데이터불러오기

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   이전   졸업논문


In [7]:
import pandas as pd
df= pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.1_증분대상_500개.csv', encoding='utf-8-sig')

In [8]:
df.head(3)

,id,document,label
0,8744926,1.전체적스토리와 주연 제외 나머지 인물들의 연기가 너무 오글오글(특히 여자주인공 ...,0
1,9797076,"지나치게 폭력적, 대사의 절반이 욕설, 알 파치노의 연기는 좋았지만 개인적으로 매력...",0
2,9305804,결말도 후지고 도입부분도 매우 그것도 아주 많이 지루하다 그리고 마지막 하일라이트 ...,0


In [9]:
# 증강 적용
df['text_swap_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_1'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_2'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_3'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_4'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

In [10]:
# 증강 적용
df['text_swap_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_5'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace_6'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

df['text_swap_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_7'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=3))
df['text_insert_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=3))
df['text_delete_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.3))
df['text_replace_8'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=3))

df['text_swap_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=4))
df['text_insert_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=4))
df['text_delete_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.4))
df['text_replace_9'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=4))



In [11]:
df.to_csv('NSMC2_0.1_aug_eda_final.csv', encoding='utf-8-sig')